In [246]:
import pandas as pd
import numpy as np 
import sys, os
import importlib.util

hp = os.path.abspath('../helper_functions.py')
spec_a = importlib.util.spec_from_file_location("helper_functions", hp)
helper_func = importlib.util.module_from_spec(spec_a)
spec_a.loader.exec_module(helper_func)

#need to get our product JSON from parent folder
p = os.path.abspath('../product_info_JSON.py')
spec = importlib.util.spec_from_file_location("product_info_JSON", p)
json_module = importlib.util.module_from_spec(spec)
spec.loader.exec_module(json_module)
#print(json_module.PRODUCT_INFORMATION)

# ---------------------------- End of Lookup Table work ---------------------------------

# Create the Abbott product table
Ncare_products_original = pd.read_csv("../../Data/Ncare/Ncare_scrape_data.csv",  usecols= json_module.PRODUCT_INFORMATION.keys()) #This could be replaced with output from other parser
Ncare_products = Ncare_products_original.copy() #to be used by future analysis
Ncare_products.drop_duplicates(keep='first', inplace=True)


In [247]:
Ncare_products.head()

,url,store,name,price,size_or_weight,availability,item_id,description,ingredients,allergin_info,...,Sizes,Form,Flavours,Case of X,Case of Y,clinical_indications,benefits,feature_table_rows,usage,entry_date
0,https://www.ncare.net.au/nutrition-products/dy...,Ncare,RESOURCE ThickenUp 1.2g,NaN,NaN,NaN,12151076,Nestle Health Science RESOURCE ThickenUp Clear...,"Maltodextrin (Corn, Potato), Thickener (Xantha...",NaN,...,NaN,NaN,Neutral,NaN,NaN,NaN,NaN,NaN,- First add desired quantity of powder into em...,27/05/2020 11:41:13
1,https://www.ncare.net.au/nutrition-products/dy...,Ncare,RESOURCE ThickenUp 125g,NaN,NaN,NaN,12132987,Nestle Health Science RESOURCE ThickenUp Clear...,"Maltodextrin (Corn, Potato), Thickener (Xantha...",NaN,...,NaN,NaN,Neutral,NaN,NaN,NaN,NaN,NaN,- Use the dosage scoop included in the tin.\n-...,27/05/2020 11:41:13
2,https://www.ncare.net.au/nutrition-products/dy...,Ncare,RESOURCE ThickenUp 900g,NaN,NaN,NaN,12114005,Nestle Health Science RESOURCE ThickenUp Clear...,"Maltodextrin (Corn, Potato), Thickener (Xantha...",NaN,...,NaN,NaN,Neutral,NaN,NaN,NaN,NaN,NaN,- Use the dosage scoop included in the tin.\n-...,27/05/2020 11:41:13
3,https://www.ncare.net.au/nutrition-products/op...,Ncare,OPTIFAST VLCD Shake 18,NaN,NaN,NaN,12317536,NaN,"Vanilla Flavour: Skimmed Milk Powder (31%), Mi...",NaN,...,NaN,NaN,"Chocolate, Vanilla",NaN,NaN,NaN,NaN,NaN,Add one sachet of OPTIFAST® VLCD™ to 200-250mL...,27/05/2020 11:41:13
4,https://www.ncare.net.au/nutrition-products/op...,Ncare,OPTIFAST VLCD Bars,NaN,NaN,NaN,12371262,These delicious and convenient Bars are one th...,"Chocolate Bar: Milk Proteins, Milk Chocolate (...",NaN,...,NaN,NaN,"Assorted, Chocolate, Berry Crunch, Cranberry, ...",NaN,NaN,NaN,NaN,NaN,Instructions\nOpen wrapper and consumer bar. O...,27/05/2020 11:41:13


In [248]:
#fix up id
Ncare_products['item_id'] = Ncare_products['item_id'].astype(str).str.replace("CASE", '', regex=False)
Ncare_products['item_id'] = Ncare_products['item_id'].astype(str).str.replace("EACH", '', regex=False)

#removing alternate name for now, better regex will fix this in the future
Ncare_products['ingredients'] = Ncare_products['ingredients'].str.replace("\\(.*?\\)", '', regex=True)
Ncare_products['ingredients'] = Ncare_products['ingredients'].str.replace(".", ',', regex=False)
Ncare_products['ingredients'] = Ncare_products['ingredients'].str.replace("May contain", '', regex=False)
Ncare_products['ingredients'] = Ncare_products['ingredients'].str.replace("Contains", '', regex=False)
#Ncare_products['ingredients']

In [249]:
#Make sure columns are matched with db
df = pd.DataFrame(columns=['item_id','ingredient'])
for i in range(len(Ncare_products)):
    row = Ncare_products.iloc[i]
    ingredients = row['ingredients'].split(',')
    ingredients = [i.strip() for i in ingredients if i != '\n ' or i != '' or i != None]
    #print(ingredients)
    item_id = row['item_id'] 
    df2 = pd.DataFrame({'item_id': [item_id]*len(ingredients), 'ingredient': ingredients} )
    df = pd.concat([df, df2])

    
df = df[df['ingredient'] != '']
df

,item_id,ingredient
0,12151076,Maltodextrin
1,12151076,Thickener
2,12151076,Mineral Salt
3,12151076,milk
0,12132987,Maltodextrin
...,...,...
10,12297832,Choline Chloride
11,12297832,Taurine
12,12297832,L-carnitine
13,12297832,Anti-Forming Agent


In [250]:
df.to_csv("../../Data/Ncare/ingredient_table.csv", index=False)

In [251]:
Ncare_products['Flavours'] = Ncare_products['Flavours'].fillna('')
Ncare_products['Flavours'].isnull().sum()

0

In [252]:


df_flav = pd.DataFrame(columns=['item_id','flavour'])
for i in range(len(Ncare_products)):
    row_f = Ncare_products.iloc[i]
    flav = (row_f['Flavours'].split(','))
    item_id = row_f['item_id']
    df2 = pd.DataFrame({'item_id': [item_id]*len(flav), 'flavour': flav} )
    df_flav = pd.concat([df_flav, df2])
    


In [253]:
df_flav.to_csv('../../Data/Ncare/flavour_table.csv', index=False)

In [254]:
Ncare_products.head()

,url,store,name,price,size_or_weight,availability,item_id,description,ingredients,allergin_info,...,Sizes,Form,Flavours,Case of X,Case of Y,clinical_indications,benefits,feature_table_rows,usage,entry_date
0,https://www.ncare.net.au/nutrition-products/dy...,Ncare,RESOURCE ThickenUp 1.2g,NaN,NaN,NaN,12151076,Nestle Health Science RESOURCE ThickenUp Clear...,"Maltodextrin , Thickener , Mineral Salt , mil...",NaN,...,NaN,NaN,Neutral,NaN,NaN,NaN,NaN,NaN,- First add desired quantity of powder into em...,27/05/2020 11:41:13
1,https://www.ncare.net.au/nutrition-products/dy...,Ncare,RESOURCE ThickenUp 125g,NaN,NaN,NaN,12132987,Nestle Health Science RESOURCE ThickenUp Clear...,"Maltodextrin , Thickener , Mineral Salt , mil...",NaN,...,NaN,NaN,Neutral,NaN,NaN,NaN,NaN,NaN,- Use the dosage scoop included in the tin.\n-...,27/05/2020 11:41:13
2,https://www.ncare.net.au/nutrition-products/dy...,Ncare,RESOURCE ThickenUp 900g,NaN,NaN,NaN,12114005,Nestle Health Science RESOURCE ThickenUp Clear...,"Maltodextrin , Thickener , Mineral Salt , mil...",NaN,...,NaN,NaN,Neutral,NaN,NaN,NaN,NaN,NaN,- Use the dosage scoop included in the tin.\n-...,27/05/2020 11:41:13
3,https://www.ncare.net.au/nutrition-products/op...,Ncare,OPTIFAST VLCD Shake 18,NaN,NaN,NaN,12317536,NaN,"Vanilla Flavour: Skimmed Milk Powder , Milk Pr...",NaN,...,NaN,NaN,"Chocolate, Vanilla",NaN,NaN,NaN,NaN,NaN,Add one sachet of OPTIFAST® VLCD™ to 200-250mL...,27/05/2020 11:41:13
4,https://www.ncare.net.au/nutrition-products/op...,Ncare,OPTIFAST VLCD Bars,NaN,NaN,NaN,12371262,These delicious and convenient Bars are one th...,"Chocolate Bar: Milk Proteins, Milk Chocolate ...",NaN,...,NaN,NaN,"Assorted, Chocolate, Berry Crunch, Cranberry, ...",NaN,NaN,NaN,NaN,NaN,Instructions\nOpen wrapper and consumer bar. O...,27/05/2020 11:41:13


In [255]:
# ------------- Setup dataframe and columns -----------------

del_cols = ['size_or_weight', 'availability', 'allergin_info', 'Case of X', 'Case of Y',
           'Flavours', 'Sizes', 'Form', 'ingredients', 'clinical_indications', 'benefits', 'feature_table_rows', 'footnotes']

for c in del_cols:
    if c in Ncare_products.columns:
        del Ncare_products[c]
        


In [256]:

del_cols = list()

for i in range(1, 4):
    del_cols.append('serving_size_' + str(i))
    del_cols.append('nutrient_table_' + str(i))
    del_cols.append('vitamin_table_' + str(i))
    del_cols.append('mineral_table_' + str(i))
    
for c in del_cols:
    if c in Ncare_products.columns:
        del Ncare_products[c]


In [257]:
Ncare_products.head()

#keeping in the columns such as price for sql writer automated (TODO: read cols to use from storage)

,url,store,name,price,item_id,description,usage,entry_date
0,https://www.ncare.net.au/nutrition-products/dy...,Ncare,RESOURCE ThickenUp 1.2g,NaN,12151076,Nestle Health Science RESOURCE ThickenUp Clear...,- First add desired quantity of powder into em...,27/05/2020 11:41:13
1,https://www.ncare.net.au/nutrition-products/dy...,Ncare,RESOURCE ThickenUp 125g,NaN,12132987,Nestle Health Science RESOURCE ThickenUp Clear...,- Use the dosage scoop included in the tin.\n-...,27/05/2020 11:41:13
2,https://www.ncare.net.au/nutrition-products/dy...,Ncare,RESOURCE ThickenUp 900g,NaN,12114005,Nestle Health Science RESOURCE ThickenUp Clear...,- Use the dosage scoop included in the tin.\n-...,27/05/2020 11:41:13
3,https://www.ncare.net.au/nutrition-products/op...,Ncare,OPTIFAST VLCD Shake 18,NaN,12317536,NaN,Add one sachet of OPTIFAST® VLCD™ to 200-250mL...,27/05/2020 11:41:13
4,https://www.ncare.net.au/nutrition-products/op...,Ncare,OPTIFAST VLCD Bars,NaN,12371262,These delicious and convenient Bars are one th...,Instructions\nOpen wrapper and consumer bar. O...,27/05/2020 11:41:13


In [258]:
Ncare_products.to_csv('../../Data/Ncare/products_table.csv', index=False)

In [259]:
import re
columns = ['item_id', 'nutrient', 'measure', 'value', 'perc_daily_recom']

#nutrition tables
full_df = pd.DataFrame(columns =  columns)
for it_id in Ncare_products['item_id'].values:
    pth = '../../Data/Ncare/Nutrition_tables/Ncare' + str(it_id) + '_nutrition_table.csv'
    Ncare_nutr = pd.read_csv(pth) 
    nutrient = Ncare_nutr.iloc[:, 1]
    nutrient_list = list()

    quantity = Ncare_nutr.iloc[:, 2]
    quantity_list = list()
    measure_list = list()
    value_list = list()

    for i in range(1, len(nutrient)):
        #print(type(nutrient.iloc[i]))
        if (isinstance(nutrient.iloc[i],str)):
            nutrient_list.append(nutrient.iloc[i].lstrip('-'))
            v = str(quantity.iloc[i]).strip()
            val = [i for i in v if (i.isnumeric()) or i == '.']
            val = ''.join(val)
            measures = ''.join(re.findall('[a-zA-Z]+',v)) 
            quantity_list.append(str(quantity.iloc[i]).strip())
            measure_list.append(measures)
            value_list.append(val)

    nutrient_list.pop(len(nutrient_list)-1)
    quantity_list.pop(len(quantity_list)-1)
    measure_list.pop(len(measure_list)-1)
    value_list.pop(len(value_list)-1)

    df = pd.DataFrame({'nutrient': nutrient_list, 'measure':measure_list,'value':value_list, 'item_id': str(it_id)})
    full_df = pd.concat([full_df, df])
                                 
full_df

,item_id,nutrient,measure,value,perc_daily_recom
0,12151076,Energy,kJ,15,NaN
1,12151076,Protein,g,0.01,NaN
2,12151076,Fat-total,g,0.0,NaN
3,12151076,Saturated,g,0.0,NaN
4,12151076,Carbohydrate,g,0.8,NaN
...,...,...,...,...,...
31,12297832,Potassium,mg,2100,NaN
32,12297832,Chromium,g,160,NaN
33,12297832,Choline,mg,800,NaN
34,12297832,Taurine,mg,200,NaN


In [260]:
full_df.to_csv('../../Data/Ncare/nutrition_table.csv', index=False)

In [261]:
import re

#nutrition tables
full_df_clin_ind = pd.DataFrame(columns = ['clinical_indication', 'item_id'])
for it_id in Ncare_products['item_id'].values:
    clin_ind = None
    pth = '../../Data/Ncare/Clinical_indications_tables/Ncare' + str(it_id) + '_clinical_indications_table.csv'
    try:
        clin_ind = pd.read_csv(pth) 
    except:
        continue
      
    clin_ind_list = list()
    for col_name in clin_ind.columns:
        if 'CLINICAL' in col_name:
            arr = clin_ind[col_name].values
            clin_ind_list = clin_ind_list + (arr).tolist()
       
   # print(clin_ind_list)
    df = pd.DataFrame({'clinical_indication': clin_ind_list, 'item_id': str(it_id)})
    full_df_clin_ind = pd.concat([full_df_clin_ind, df])
                                 
full_df_clin_ind

,clinical_indications,item_id
0,Burns,12147940
1,Dialysis,12147940
2,Food fortification,12147940
3,Volume sensitive or restricted patients,12147940
4,Cancer,12147940
...,...,...
3,NaN,12370717
4,Inadequate oral intake,12370717
5,For the management of diarrhoea,12370717
6,NaN,12370717


In [262]:
full_df_clin_ind.to_csv('../../Data/Ncare/clinical_indications_table.csv', index=False)